In [1]:
import keras

2023-01-11 14:35:16.662870: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 14:35:17.077538: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.2/lib64
2023-01-11 14:35:17.077651: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 14:35:19.060507: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open share

### 이미지 높이 400으로 거정, 비율이 맞게끔 너비 변환

In [2]:
from tensorflow.keras.utils import load_img, img_to_array, save_img

# 변환하려는 이미지 경로
target_image_path = './protrait.jpg'

# 스타일 이미지 경로
style_reference_image_path = './popova.jpg'

#생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)    # 비율에 맞춰서 줄여준다

### VGG19 사용

In [9]:
import numpy as np
# from keras.applications import vgg19
from tensorflow.keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # VGG19.preprocessing_input 함수에서 일어나는 변환을 복원하기 위해 ImageNet의 평균 픽셀 값을 더하고
    
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # 'BGR' -> 'RGB' 로 변환
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### VGG19 네트워크 설정

### input_tensor는 타깃이미지, 스타일이미지, 생성된 이미지를 행으로 쌓게됨
### 결국 input_tensor의 차원은 (3, 400, width, 3)이 됨

In [19]:
# from keras import backend as K
from tensorflow.keras import backend as K

# 이미 준비되어 있는 이미지이므로  constant를 이용해서 정의
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합침
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듦
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료')

모델 로드 완료


### 콘텐츠 손실과 스타일 손실을 정의

In [20]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

### 두 손실 함수에 하나의 손실함수를 더 사용

In [22]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))